In [ ]:
import sympy as sp

V_m, R, T, a, b = sp.symbols('V_m R T a b')
P = sp.symbols('P')

# Peng-Robinson equation of state
P_eq = (R * T / (V_m - b)) - (a / (V_m * (V_m + b) + b * (V_m - b)))

integral = sp.integrate(P_eq, V_m)

integral_result = sp.simplify(integral)
integral_result




In [ ]:
# Isothermal Stirling engine
def carnot_eff(hot_source_temp: float, cold_source_temp: float) -> float:
    return 1.0 - cold_source_temp/hot_source_temp

speed = 1.0
load = 1.0

mass = 1
pressure = 10

iter_count = 0
max_iter = 100

T_h = 500           # Temperature of the hot source
T_k = 200           # Temperature of the cold source

while (iter_count < max_iter):

    iter_count += 1